In [108]:
#API key - eq63r7svaqsta7wm_6bzrykdx2zj4jfnh
from san import *
import san
from decimal import Decimal
import pandas as pd
import datetime

In [48]:
#Getting all slugs/tickers/names/marketsegment/supply and dropping all with missing or 0 supply
data = san.get("projects/all")
data = data.dropna()
data['totalSupply'] = data['totalSupply'].astype(float)
data = data[data['totalSupply'] != 0]

In [58]:
#limiting only to cryptocurrency
crypto = data[data["marketSegment"] == "Cryptocurrency"]
crypto = crypto.sort_values(by = ["totalSupply"], ascending = False)
crypto

,marketSegment,name,slug,ticker,totalSupply
1748,Cryptocurrency,Dimecoin,dimecoin,DIME,5.559981e+11
1181,Cryptocurrency,Mooncoin,mooncoin,MOON,2.274909e+11
1854,Cryptocurrency,NewYorkCoin,newyorkcoin,NYC,1.437611e+11
1707,Cryptocurrency,Dogecoin,dogecoin,DOGE,1.291275e+11
1769,Cryptocurrency,eXPerience Chain,experience-chain,XPC,1.132155e+11
...,...,...,...,...,...
1442,Cryptocurrency,ParallelCoin,parallelcoin,DUO,3.147186e+05
347,Cryptocurrency,Unobtanium,unobtanium,UNO,2.033284e+05
238,Cryptocurrency,Wrapped Bitcoin,wrapped-bitcoin,WBTC,1.459507e+05
1761,Cryptocurrency,Amoveo,amoveo,VEO,6.983818e+04


In [142]:
#Daily price list definition
#Definition will create daily return as percentage
def pricelist(slug):
    fullname = "price_usd/" + slug
    prices = san.get(
    fullname,
    from_date="2018-04-01",
    to_date="2021-04-01",
    interval="1d"
    )
    newname = slug+"_price"
    finaldf = prices.rename(columns={"value": newname})#For some reason it won't let me rename
    finaldf.index.name = 'datetime'
    finaldf.reset_index(inplace=True)
    finaldf['lagprice'] = finaldf[newname].shift(1)
    dr_slug = "dret_"+slug
    finaldf[dr_slug] = (finaldf[newname] - finaldf['lagprice'])/finaldf['lagprice']
    finaldf = finaldf.drop(['lagprice'], axis = 1)
    #note - right now I only care about daily return take this out if you want price data
    finaldf = finaldf.drop([newname], axis=1)
    return(finaldf)

In [189]:
#Running through all cryptocurrencies and adding price and daily returns
count = 0
allpdret = pd.DataFrame()
for i in crypto['slug'].tolist():
    holddf = pricelist(i)
    if count == 0:
        allpdret = holddf
    else:
        allpdret = pd.merge(allpdret, holddf, how = "left", on = "datetime")
    count = count+1

In [196]:
#Creating daily equal weighted market return (not really interested in value weight right now)

#will be used to calculate sum
temphold = allpdret[:]
temphold = temphold.fillna(0)
temphold = temphold.drop(['datetime'], axis = 1)
temphold['sumreturn'] = temphold.sum(axis = 1)
stuff1 = temphold['sumreturn'].tolist()

#will hold count of firms per year that have non-na returns
temp2hold = allpdret[:]
temp2hold = temp2hold.drop(['datetime'], axis = 1)
temp2hold['count'] = temp2hold.count(axis=1)
stuff2 = temp2hold['count'].tolist()

#will be used to calculate daily return
temp3hold = allpdret[:]
temp3hold['sumreturn'] = stuff1
temp3hold['count'] = stuff2
temp3hold['ew_avgret'] = temp3hold['sumreturn']/temp3hold['count']
ew_avgret = temp3hold['ew_avgret'].tolist()

#Can add value weighted with market cap

C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [200]:
#Bitcoin return only - can create definition on slug to find every one if necessary
slug = "bitcoin"
dretname = "dret_" + slug
slug_data = allpdret[["datetime", dretname]]
slug_data['ew_market'] = ew_avgret
slug_data['excess_return'] = slug_data[dretname] - slug_data['ew_market']

C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [202]:
slug_data

,datetime,dret_bitcoin,ew_market,excess_return
0,2018-04-01 00:00:00+00:00,NaN,NaN,NaN
1,2018-04-02 00:00:00+00:00,0.035003,0.128587,-0.093584
2,2018-04-03 00:00:00+00:00,0.052558,0.087226,-0.034669
3,2018-04-04 00:00:00+00:00,-0.080775,-0.087880,0.007105
4,2018-04-05 00:00:00+00:00,-0.006182,0.026519,-0.032701
...,...,...,...,...
1092,2021-03-28 00:00:00+00:00,-0.000407,-0.000295,-0.000112
1093,2021-03-29 00:00:00+00:00,0.032161,0.138622,-0.106460
1094,2021-03-30 00:00:00+00:00,0.020216,0.049555,-0.029339
1095,2021-03-31 00:00:00+00:00,0.000019,0.112242,-0.112222
